## PySpark 설치

In [1]:
!pip install pyspark==3.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840629 sha256=62f2b80ceac3a9139147726b481336f0c12d24b3595abdf34b3d6c0024cba3dd
  Stored in directory: /root/.cache/pip/wheels/07/a0/a3/d24c94bf043ab5c7e38c30491199a2a11fef8d2584e6df7fb7
Successfully built pyspark
  Attempting uninstall: pyspark
    Found existing installation: pyspark 3.5.1
    Uninstalling pyspark-3.5.1:
      Successfully uninstalled pyspark-3.5.1


In [2]:
!cd /usr/local/lib/python3.12/dist-packages/pyspark/jars && wget https://s3.amazonaws.com/redshift-downloads/drivers/jdbc/2.1.0.30/redshift-jdbc42-2.1.0.30.jar

--2025-11-21 07:00:57--  https://s3.amazonaws.com/redshift-downloads/drivers/jdbc/2.1.0.30/redshift-jdbc42-2.1.0.30.jar
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.121.224, 52.217.104.238, 52.217.139.64, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.121.224|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1088404 (1.0M) [application/java-archive]
Saving to: ‘redshift-jdbc42-2.1.0.30.jar’

redshift-jdbc42-2.1 100%[===================>]   1.04M  6.73MB/s    in 0.2s    

2025-11-21 07:00:57 (6.73 MB/s) - ‘redshift-jdbc42-2.1.0.30.jar’ saved [1088404/1088404]



In [3]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL #1") \
    .config("spark.jars", "/usr/local/lib/python3.10/dist-packages/pyspark/jars/redshift-jdbc42-2.1.0.30.jar") \
    .getOrCreate()

## Redshift 상의 다음 테이블을 이용하여 월별 채널별 매출과 방문자 정보 계산하기
user_session_channel, session_timestamp, session_transaction

In [4]:
# Redshift와 연결해서 DataFrame으로 로딩하기
url = "jdbc:redshift://learnde.cduaw970ssvt.ap-northeast-2.redshift.amazonaws.com:5439/dev?user=your-username&password=your-password"

df_user_session_channel = spark.read \
    .format("jdbc") \
    .option("driver", "com.amazon.redshift.Driver") \
    .option("url", url) \
    .option("dbtable", "raw_data.user_session_channel") \
    .load()

df_session_timestamp = spark.read \
    .format("jdbc") \
    .option("driver", "com.amazon.redshift.Driver") \
    .option("url", url) \
    .option("dbtable", "raw_data.session_timestamp") \
    .load()

df_session_transaction = spark.read \
    .format("jdbc") \
    .option("driver", "com.amazon.redshift.Driver") \
    .option("url", url) \
    .option("dbtable", "raw_data.session_transaction") \
    .load()

In [5]:
df_user_session_channel.createOrReplaceTempView("user_session_channel")
df_session_timestamp.createOrReplaceTempView("session_timestamp")
df_session_transaction.createOrReplaceTempView("session_transaction")

In [6]:
spark.table("session_timestamp").show(5)

+--------------------+--------------------+
|           sessionid|                  ts|
+--------------------+--------------------+
|00029153d12ae1c9a...|2019-10-18 14:14:...|
|0004289ee1c7b8b08...|2019-11-16 21:20:...|
|0006246bee639c7a7...|2019-08-10 16:33:...|
|0006dd05ea1e999dd...|2019-07-06 19:54:...|
|000958fdaefe0dd06...|2019-11-02 14:52:...|
+--------------------+--------------------+
only showing top 5 rows



In [7]:
df_user_session_channel.show(5)

+------+--------------------+---------+
|userid|           sessionid|  channel|
+------+--------------------+---------+
|  1651|0004289ee1c7b8b08...|  Organic|
|  1197|00053f5e11d1fe4e4...| Facebook|
|  1401|00056c20eb5a02958...| Facebook|
|  1399|00063cb5da1826feb...| Facebook|
|  1667|000958fdaefe0dd06...|Instagram|
+------+--------------------+---------+
only showing top 5 rows



In [8]:
df_session_timestamp.show(5)

+--------------------+--------------------+
|           sessionid|                  ts|
+--------------------+--------------------+
|00029153d12ae1c9a...|2019-10-18 14:14:...|
|0004289ee1c7b8b08...|2019-11-16 21:20:...|
|0006246bee639c7a7...|2019-08-10 16:33:...|
|0006dd05ea1e999dd...|2019-07-06 19:54:...|
|000958fdaefe0dd06...|2019-11-02 14:52:...|
+--------------------+--------------------+
only showing top 5 rows



In [9]:
df_session_transaction.show(5)

+--------------------+--------+------+
|           sessionid|refunded|amount|
+--------------------+--------+------+
|00029153d12ae1c9a...|   false|    85|
|008909bd27b680698...|   false|    13|
|0107acb41ef20db22...|   false|    16|
|018544a2c48077d2c...|   false|    39|
|020c38173caff0203...|   false|    61|
+--------------------+--------+------+
only showing top 5 rows



## 월별 채널별 총 방문자와 매출 발생 방문자 계산

In [10]:
# JOIN key가 정말 하나씩만 존재하나?
spark.sql("""SELECT sessionid, COUNT(1) count
FROM user_session_channel
GROUP BY 1
ORDER BY 2 DESC
LIMIT 1""").show()

+--------------------+-----+
|           sessionid|count|
+--------------------+-----+
|0135456d6a3c1051f...|    1|
+--------------------+-----+



In [11]:
# JOIN key가 정말 하나씩만 존재하나?
spark.sql("""SELECT sessionid, COUNT(1) count
FROM session_transaction
GROUP BY 1
ORDER BY 2 DESC
LIMIT 1""").show()

+--------------------+-----+
|           sessionid|count|
+--------------------+-----+
|532ff98823e7d1433...|    1|
+--------------------+-----+



In [12]:
# JOIN key가 정말 하나씩만 존재하나?
spark.sql("""SELECT sessionid, COUNT(1) count
FROM session_timestamp
GROUP BY 1
ORDER BY 2 DESC
LIMIT 1""").show()

+--------------------+-----+
|           sessionid|count|
+--------------------+-----+
|027d77b7d744e5647...|    1|
+--------------------+-----+



 ### 월별 채널별 총 방문자 계산

In [13]:
mon_channel_rev_df = spark.sql("""
    SELECT LEFT(sti.ts, 7) year_month,
      usc.channel channel,
      COUNT(DISTINCT userid) total_visitors
    FROM user_session_channel usc
    LEFT JOIN session_timestamp sti ON usc.sessionid = sti.sessionid
    GROUP BY 1 ,2
    ORDER BY 1, 2""")

In [14]:
mon_channel_rev_df.show()

+----------+---------+--------------+
|year_month|  channel|total_visitors|
+----------+---------+--------------+
|   2019-05| Facebook|           247|
|   2019-05|   Google|           253|
|   2019-05|Instagram|           234|
|   2019-05|    Naver|           237|
|   2019-05|  Organic|           238|
|   2019-05|  Youtube|           244|
|   2019-06| Facebook|           414|
|   2019-06|   Google|           412|
|   2019-06|Instagram|           410|
|   2019-06|    Naver|           398|
|   2019-06|  Organic|           416|
|   2019-06|  Youtube|           400|
|   2019-07| Facebook|           558|
|   2019-07|   Google|           556|
|   2019-07|Instagram|           567|
|   2019-07|    Naver|           553|
|   2019-07|  Organic|           557|
|   2019-07|  Youtube|           564|
|   2019-08| Facebook|           611|
|   2019-08|   Google|           610|
+----------+---------+--------------+
only showing top 20 rows



 ### 월별 채널별 총 방문자와 구매 방문자 계산

In [15]:
mon_channel_rev_df = spark.sql("""
    SELECT LEFT(sti.ts, 7) year_month,
      usc.channel channel,
      COUNT(DISTINCT userid) total_visitors,
      COUNT(DISTINCT CASE WHEN amount is not NULL THEN userid END) paid_visitors
    FROM user_session_channel usc
    LEFT JOIN session_timestamp sti ON usc.sessionid = sti.sessionid
    LEFT JOIN session_transaction str ON usc.sessionid = str.sessionid
    GROUP BY 1 ,2
    ORDER BY 1, 2""")

In [16]:
mon_channel_rev_df.show()

+----------+---------+--------------+-------------+
|year_month|  channel|total_visitors|paid_visitors|
+----------+---------+--------------+-------------+
|   2019-05| Facebook|           247|           14|
|   2019-05|   Google|           253|           10|
|   2019-05|Instagram|           234|           11|
|   2019-05|    Naver|           237|           11|
|   2019-05|  Organic|           238|           17|
|   2019-05|  Youtube|           244|           10|
|   2019-06| Facebook|           414|           22|
|   2019-06|   Google|           412|           13|
|   2019-06|Instagram|           410|           21|
|   2019-06|    Naver|           398|           15|
|   2019-06|  Organic|           416|           14|
|   2019-06|  Youtube|           400|           17|
|   2019-07| Facebook|           558|           32|
|   2019-07|   Google|           556|           21|
|   2019-07|Instagram|           567|           25|
|   2019-07|    Naver|           553|           19|
|   2019-07|

## 월별 채널별 총 매출액 (리펀드 포함), 총 방문자, 매출 발생 방문자, 전환률 계산

In [17]:
mon_channel_rev_df = spark.sql("""
  SELECT LEFT(ts, 7) month,
       usc.channel,
       COUNT(DISTINCT userid) uniqueUsers,
       COUNT(DISTINCT (CASE WHEN amount >= 0 THEN userid END)) paidUsers,
       SUM(amount) grossRevenue,
       SUM(CASE WHEN refunded is not True THEN amount END) netRevenue,
       ROUND(COUNT(DISTINCT (CASE WHEN amount >= 0 THEN userid END)) * 100
          / COUNT(DISTINCT userid), 2) conversionRate
   FROM user_session_channel usc
   LEFT JOIN session_timestamp t ON t.sessionid = usc.sessionid
   LEFT JOIN session_transaction st ON st.sessionid = usc.sessionid
   GROUP BY 1, 2
   ORDER BY 1, 2;
""")

In [18]:
mon_channel_rev_df.show()

+-------+---------+-----------+---------+------------+----------+--------------+
|  month|  channel|uniqueUsers|paidUsers|grossRevenue|netRevenue|conversionRate|
+-------+---------+-----------+---------+------------+----------+--------------+
|2019-05| Facebook|        247|       14|        1199|       997|          5.67|
|2019-05|   Google|        253|       10|         580|       580|          3.95|
|2019-05|Instagram|        234|       11|         959|       770|           4.7|
|2019-05|    Naver|        237|       11|         867|       844|          4.64|
|2019-05|  Organic|        238|       17|        1846|      1571|          7.14|
|2019-05|  Youtube|        244|       10|         529|       529|           4.1|
|2019-06| Facebook|        414|       22|        1578|      1578|          5.31|
|2019-06|   Google|        412|       13|         947|       947|          3.16|
|2019-06|Instagram|        410|       21|        1462|      1418|          5.12|
|2019-06|    Naver|        3